# Analyze users by inspecting retweet counts

In [ ]:
import pandas as pd
from IPython.display import clear_output

In [ ]:
DATADIR = "/home/erikt/projects/puregome/data/retweets/"
DATAFILE = "get-retweet-users.py.out.gz"
RIGHT = "geertwilderspvv thierrybaudet fvdemocratie".split(" ")
DUTCHPRESS = "telegraaf NOS RTLnieuws NUnl ADnl nrc volkskrant trouw parool AT5".split(" ")
FLEMISHPRESS = "HLN_BE demorgen destandaard vrtnws Nieuwsblad_be".split(" ")

In [ ]:
def squeal(text=None):
    clear_output(wait=True)
    if not text is None: print(text)

In [ ]:
df = pd.read_csv(DATADIR+DATAFILE,sep=" ",header=None,compression="gzip")

In [ ]:
groups = df.groupby([0,1]).groups

In [ ]:
profiles = {}

def add(user,group,count):
    if not user in profiles: profiles[user] = {RIGHT[0]:0,DUTCHPRESS[0]:0,FLEMISHPRESS[0]:0}
    if not group[0] in profiles[user]: profiles[user][group[0]] = count
    else: profiles[user][group[0]] += count

counter = 0
for key in groups.keys():
    counter += 1
    if counter % 10000 == 0: squeal(str(counter)+"/"+str(len(groups)))
    retweetedUser = key[1]
    retweeter = key[0]
    if retweetedUser in RIGHT: add(retweeter,RIGHT,len(groups[key]))
    elif retweetedUser in DUTCHPRESS: add(retweeter,DUTCHPRESS,len(groups[key]))
    elif retweetedUser in FLEMISHPRESS: add(retweeter,FLEMISHPRESS,len(groups[key]))

In [ ]:
TARGET = FLEMISHPRESS[0]
sortedP = {key:profiles[key] for key in sorted(profiles.keys(),key=lambda k:profiles[k][TARGET],reverse=True) if TARGET in profiles[key]}
selectedP = {key:sortedP[key] for key in sortedP if sortedP[key][TARGET] > 0 and \
                                                    sortedP[key][RIGHT[0]] == 0 and \
                                                    sortedP[key][DUTCHPRESS[0]] == 0}
results = [df.iloc[i][1] for i in range(0,len(df)) if df.iloc[i][0] in selectedP.keys()]

In [ ]:
groupsF = pd.DataFrame(results).groupby(0).groups
{key:len(groupsF[key]) for key in sorted(groupsF.keys(),key=lambda k:len(groupsF[k]),reverse=True)}